<font size=5 color="black">**任務流程**</font>
1. 前置作業 ＝> import 需要套件、建立相關身份認證資料
2. 打開開發人員工具，找到搜尋html，使用selenium輸入關鍵字搜尋，並進入搜尋結果頁面（詳情：selenium-dcard應用），104為動態網頁，使用selenium進行頁面滑動，依個人需求取得各職缺網址，並存成一個網址list
4. 透過迴圈進入各職缺網址內，取得相關資料，append進入output_list中，後續將此list轉成df
5. 將爬蟲回來的資料存成df，進行data cleaning，完成後輸出為excel檔！

參考資料:
<br>
1. [104人力銀行 網路爬蟲](https://tlyu0419.github.io/2020/06/19/Crawler-104HumanResource/)
<br>
2. [[網頁爬蟲] 104人力銀行標籤抓不到內容](https://ithelp.ithome.com.tw/questions/10198403)
<br>
3. [Python 使用 Beautiful Soup 抓取與解析網頁資料，開發網路爬蟲教學](https://blog.gtwang.org/programming/python-beautiful-soup-module-scrape-web-pages-tutorial/2/)

### 前置作業 ＝> import 需要套件、建立相關身份認證資料
1. 設置optins => 擋掉跳出視窗
2. 建立headers
3. 準備參數cookies (全職、關鍵字、清單條例式)

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import time
import re

In [3]:
options = Options()
options.add_argument("--disable-notifications")

url_search = "https://104.com.tw/jobs/search/?"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
my_params = {'ro': '1',  # 限定全職的工作，如果不限定則輸入0
             'keyword': '數據分析',     # 想要查詢的關鍵字
             'mode': 'l'}   # 清單瀏覽模式


### 打開開發人員工具，找到搜尋html，使用selenium輸入關鍵字搜尋，並進入搜尋結果頁面

1. session連線 => 帶著參數通過搜尋頁面，抵達結果頁面的網址，104為動態網頁，使用selenium進行頁面滑動
2. 輸入欲爬取至第幾頁面
3. 建立url的beautiful soup物件，取得各職缺網址並存成work_url_lists
    - 使用**driver.page_source**來取得所有頁面資訊
    - 過濾掉置頂徵才廣告 => url中包含hotjob_chr參數
    - 取得url中的職缺編號，並組成手機版網址 =>　https://m.104.com.tw/job/ + 職缺編號(ex. 5944x)

In [4]:
# session 連線
# 帶著參數通過搜尋頁面，抵達結果頁面的網址
ss = requests.session()
res = ss.get(url_search, headers=headers, params=my_params)

# 104為動態網頁，使用selenium進行頁面滑動，蒐集網址
# 輸入欲爬取至第幾頁面
page_number = int(input("請輸入欲爬取至第幾頁面: "))
driver = Chrome("../../chromedriver", options=options)
driver.get(res.url)
for i in range(1, page_number+1):
    driver.execute_script("var s = document.documentElement.scrollTop=5000")
    print("目前爬至第{}頁".format(i))
    time.sleep(5)

# 建立beautiful soup物件
# 使用driver.page_source來取得所有頁面資訊
# 過濾掉置頂徵才廣告 => url中包含hotjob_chr參數
# 取得url中的職缺編號，並組成手機版網址 =>　https://m.104.com.tw/job/ + 職缺編號(ex. 5944x)
soup = BeautifulSoup(driver.page_source, features='html.parser')
url_data = [url["href"] for url in soup.select('li.job-mode__jobname a') if url["href"].split("=")[1] != 'hotjob_chr']
work_url_lists = list(map(lambda url: 'https://m.104.com.tw/job/{}'.format(re.findall("job/(.*)\?", url)[0]), url_data))

# 關閉頁面
driver.close()

print(work_url_lists)

請輸入欲爬取至第幾頁面: 4
目前爬至第1頁
目前爬至第2頁
目前爬至第3頁
目前爬至第4頁
['https://m.104.com.tw/job/5944x', 'https://m.104.com.tw/job/498rd', 'https://m.104.com.tw/job/69nkz', 'https://m.104.com.tw/job/6sxgk', 'https://m.104.com.tw/job/6sg76', 'https://m.104.com.tw/job/7aipl', 'https://m.104.com.tw/job/3g1gd', 'https://m.104.com.tw/job/6yu84', 'https://m.104.com.tw/job/7a7nd', 'https://m.104.com.tw/job/758eh', 'https://m.104.com.tw/job/799p1', 'https://m.104.com.tw/job/76g3s', 'https://m.104.com.tw/job/793ta', 'https://m.104.com.tw/job/78bew', 'https://m.104.com.tw/job/79zig', 'https://m.104.com.tw/job/6j51q', 'https://m.104.com.tw/job/6nqcb', 'https://m.104.com.tw/job/7a7n3', 'https://m.104.com.tw/job/72qxg', 'https://m.104.com.tw/job/70c5w', 'https://m.104.com.tw/job/76f27', 'https://m.104.com.tw/job/6vq61', 'https://m.104.com.tw/job/6az1x', 'https://m.104.com.tw/job/6th3c', 'https://m.104.com.tw/job/6rglt', 'https://m.104.com.tw/job/78f3u', 'https://m.104.com.tw/job/7a2jf', 'https://m.104.com.tw/job/779n8', 

### 透過迴圈進入各職缺網址內，取得相關資料，append進入output中，後續將此轉成df
1. 每5秒爬一次，避免被鎖或是重導到電腦版網址，造成錯誤發生
2. 取得職缺相關資料
    - 更新時間、公司、職位、工作地點、工作內容
    - 工作需求技能、工具、其他條件
    - 額外資訊(聯絡人、薪水)
5. append進入output

In [5]:
# 每5秒爬一次，避免被鎖或是導向網頁，造成錯誤發生
output = []
for work_url in work_url_lists:
    print(work_url)
    res_work = ss.get(work_url, headers=headers, allow_redirects=False)
    w_soup = BeautifulSoup(res_work.text, 'html.parser')
    
    try:
        # 更新時間、公司、職位、工作地點、工作內容
        update_time = w_soup.select("time")[0]['datetime']
        company = w_soup.select('h2.company')[0].text
        title = w_soup.select('h1.title')[0].text
        work_place = w_soup.select('div.content a.addr')[0].text.strip()
        description = w_soup.select("div.content p")[0].text.strip()
        
        # 工作需求技能、工具、其他條件
        job_skill = " ".join([i.text for i in w_soup.select("a[data-gtm='job-skill']")])
        job_tool = " ".join([i.text for i in w_soup.select("a[data-gtm='job-tool']")])
        if w_soup.select("td div.cut"):
            plus = w_soup.select("td div.cut")[0].text
        else:
            plus = ""
        
        ### 額外資訊(聯絡人、薪水)
        recruiter = w_soup.find_all("table", class_="column2 contact")[0].select("td")[0].text
        extra_data = ''.join([t.text for t in w_soup.select("table.column2 td")])
        salary = re.findall(r"待遇面議.*|[年月]薪.*[元以上]", extra_data) 
        
    except ValueError as e:
        print("Value error", work_url)
    except IndexError as e:
        print("Index error", work_url)
        
    output.append([update_time, company, title, work_place, salary, description, job_skill, job_tool, plus, recruiter, work_url])
    
    time.sleep(3)


https://m.104.com.tw/job/5944x
https://m.104.com.tw/job/498rd
https://m.104.com.tw/job/69nkz
https://m.104.com.tw/job/6sxgk
https://m.104.com.tw/job/6sg76
https://m.104.com.tw/job/7aipl
https://m.104.com.tw/job/3g1gd
https://m.104.com.tw/job/6yu84
https://m.104.com.tw/job/7a7nd
https://m.104.com.tw/job/758eh
https://m.104.com.tw/job/799p1
https://m.104.com.tw/job/76g3s
https://m.104.com.tw/job/793ta
https://m.104.com.tw/job/78bew
https://m.104.com.tw/job/79zig
https://m.104.com.tw/job/6j51q
https://m.104.com.tw/job/6nqcb
https://m.104.com.tw/job/7a7n3
https://m.104.com.tw/job/72qxg
https://m.104.com.tw/job/70c5w
https://m.104.com.tw/job/76f27
https://m.104.com.tw/job/6vq61
https://m.104.com.tw/job/6az1x
https://m.104.com.tw/job/6th3c
https://m.104.com.tw/job/6rglt
https://m.104.com.tw/job/78f3u
https://m.104.com.tw/job/7a2jf
https://m.104.com.tw/job/779n8
https://m.104.com.tw/job/6psms
https://m.104.com.tw/job/6uj8e
https://m.104.com.tw/job/6bcn9
https://m.104.com.tw/job/74t7v
https://

### 將爬蟲回來的資料存成df，進行data cleaning
1. 設置欄位
2. 去除重複值 => 透過company, title, work_place來篩選，保留第一個(日期較新)
3. 建立將df備份成df2，避免資料處理中發生錯誤改動到原檔而重新爬過一次

In [6]:
df = pd.DataFrame(output, columns=['update_time', 'company', 'title', 'work_place', 'salary', 'description', 'job_skill', 'job_tool', "plus", "recruiter", "URL"])

In [7]:
df.shape

(150, 11)

In [8]:
df2 = df.copy()

#### 去除重複值 => 透過company, title, work_place來篩選掉相同重複值缺，保留第一個(日期較新)

In [9]:
df2.drop_duplicates(subset=['company', 'title', 'work_place'], keep="first", inplace=True)
df2.sort_values(by="update_time", ascending=False, inplace=True)
df2

,update_time,company,title,work_place,salary,description,job_skill,job_tool,plus,recruiter,URL
0,2021-05-27,大數據股份有限公司,數據分析師,台北市中山區 ...,"[月薪 35,000~50,000元]",茫茫網海中，誰是KOL關鍵領袖，引領著話題風向？\rFB數據中，哪個分眾才是主要品牌的目標族...,市場調查企劃與執行 市場調查資料分析與報告撰寫 統計軟體操作 調查樣本統計分析,,\n 有一年以上工作經驗\r具備資料庫分析經驗...,林小姐,https://m.104.com.tw/job/5944x
127,2021-05-27,OB嚴選_橘熊科技股份有限公司,行銷數據分析專員-行銷整合處,台北市南港區八德路四段768巷1弄18號13樓 ...,[待遇面議 ...,1.數據相關專案規劃與執行。2.因應公司行銷策略進行相關規劃及與跨部門溝通協調達成目標。3....,,Python R Tableau,\n - 對於使用行銷自動化工具有高度熱忱 (...,人資部,https://m.104.com.tw/job/7aak3
125,2021-05-27,歐愛兒股份有限公司,(資深) 數據分析師 (儲備幹部),台北市大安區新生南路三段88號6樓 ...,[待遇面議 ...,1. 搜集市場情報及對銷售數字，數據資料清洗與統計分析。 2. 運用演算法建立模型及優化，整...,財務及營業分析 市場調查資料分析與報告撰寫 系統整合分析 資料庫系統管理維護 資料庫軟體應用,Python MS SQL,,HR,https://m.104.com.tw/job/77hna
121,2021-05-27,新加坡商蝦皮娛樂電商有限公司台灣分公司,Business Analyst 商業分析師 / Business Intelligence...,台北市信義區菸廠路88號,[待遇面議 ...,Business Intelligence business analyst ensures...,,MS SQL Excel Project Tableau Power BI​,\n Basic Requirements- ...,Human Resource,https://m.104.com.tw/job/7agz6
30,2021-05-27,燦坤實業股份有限公司,總部-行銷策略部-CRM數據分析資深專員,台北市內湖區堤頂大道一段331號5樓,"[月薪 38,000~50,000元]",【工作職掌】1. CRM數據分析與運用2. CRM操作效益追蹤與提升CRM價值3. 進行會員...,統計軟體操作,,\n 1.主動積極，具數據研究分析熱誠。數據解...,曾先生,https://m.104.com.tw/job/6bcn9
...,...,...,...,...,...,...,...,...,...,...,...
72,2021-04-23,華碩電腦股份有限公司,AD30084 數據分析運用工程師,台北市北投區 ...,"[月薪 30,000~60,000元]",1. 具有網路基礎架構與網路服務常識2. 具有 Window、Linux基本管理能力3. 具...,,hadoop,\n 1. Linux各平台操作(RHEL、C...,人資處招募任用部,https://m.104.com.tw/job/6x5r9
54,2021-04-22,中華電信關係企業_宏華國際股份有限公司,中華電信子公司-數據分析人員,新北市板橋區民族路168號,"[月薪 35,000~50,000元]",1.分析業務數據及撰寫簡報。2.協助規劃各項獎金與行銷制度。3.利用資料處理工具製作與整理報...,,Access Excel PowerPoint Power BI​,\n 1.具1年以上數據分析或專案管理相關經驗...,招募任用部,https://m.104.com.tw/job/79ilu
47,2021-04-21,精誠資訊股份有限公司,數據分析顧問(B216),台北市內湖區瑞光路318號 ...,[待遇面議 ...,工作內容我們是Tableau、Alteryx與Siemens MindSphere等產品的代...,專案規劃執行╱範圍管理 專案溝通╱整合管理 資料庫軟體應用 MES工廠執行系統規劃管理 系統...,Systems Analysis Systems Analyst Data Modeling...,\n,人力資源部,https://m.104.com.tw/job/6n0rh
79,2021-03-29,台灣之星電信股份有限公司,07_用戶行為數據分析師,新北市中和區建一路186號,"[月薪 30,000~44,000元]",**40%數據分析1.對發現或挖掘到的問題具有敏銳的觀察力，掌握各種演算法的使用時機2.以開...,,,\n 1. 大學以上畢業2. 熟悉 R 或 P...,人力資源部,https://m.104.com.tw/job/4m2t0


#### 薪資(salary)
將薪資進行統一格式處理
- 分成下限 & 上限 => 以利後續查找資料
- 年薪改為月薪 => 除以12
- 待遇面議改為下限4w、上限面議



In [10]:
df2["salary"] = df2["salary"].apply(lambda x: re.sub(" ", "", x[0]))

In [11]:
df2.loc[(df2["salary"].str.contains("待遇面議")), "salary"] = "40000~0"

In [12]:
df2["salary"] = df2["salary"].apply(lambda x: re.sub(",|[^0-9~]", "", x)) 

In [13]:
# 有些只寫起薪的會造成error，index用-1避免報錯
df2["salary_min"] = df2["salary"].map(lambda x: x.split("~")[0])
df2["salary_max"] = df2["salary"].map(lambda x: x.split("~")[-1])

In [14]:
df2[["salary_min", "salary_max"]] = df2[["salary_min", "salary_max"]].apply(pd.to_numeric)

In [15]:
## 年薪改為月薪
cond_low = df2["salary_min"].mean() + 3*df2["salary_min"].std()
cond_upper = df2["salary_max"].mean() + 3*df2["salary_max"].std()

df2["salary_min"] = df2["salary_min"].apply(lambda x: int(x/12) if x > cond_low else x)
df2["salary_max"] = df2["salary_max"].apply(lambda x:int(x/12) if x >cond_upper else x)

In [16]:
df2.loc[(df2["salary_min"] == df2["salary_max"]) | (df2["salary_max"] == 0), "salary_max"] = "面議"

In [17]:
df2

,update_time,company,title,work_place,salary,description,job_skill,job_tool,plus,recruiter,URL,salary_min,salary_max
0,2021-05-27,大數據股份有限公司,數據分析師,台北市中山區 ...,35000~50000,茫茫網海中，誰是KOL關鍵領袖，引領著話題風向？\rFB數據中，哪個分眾才是主要品牌的目標族...,市場調查企劃與執行 市場調查資料分析與報告撰寫 統計軟體操作 調查樣本統計分析,,\n 有一年以上工作經驗\r具備資料庫分析經驗...,林小姐,https://m.104.com.tw/job/5944x,35000,50000
127,2021-05-27,OB嚴選_橘熊科技股份有限公司,行銷數據分析專員-行銷整合處,台北市南港區八德路四段768巷1弄18號13樓 ...,40000~0,1.數據相關專案規劃與執行。2.因應公司行銷策略進行相關規劃及與跨部門溝通協調達成目標。3....,,Python R Tableau,\n - 對於使用行銷自動化工具有高度熱忱 (...,人資部,https://m.104.com.tw/job/7aak3,40000,面議
125,2021-05-27,歐愛兒股份有限公司,(資深) 數據分析師 (儲備幹部),台北市大安區新生南路三段88號6樓 ...,40000~0,1. 搜集市場情報及對銷售數字，數據資料清洗與統計分析。 2. 運用演算法建立模型及優化，整...,財務及營業分析 市場調查資料分析與報告撰寫 系統整合分析 資料庫系統管理維護 資料庫軟體應用,Python MS SQL,,HR,https://m.104.com.tw/job/77hna,40000,面議
121,2021-05-27,新加坡商蝦皮娛樂電商有限公司台灣分公司,Business Analyst 商業分析師 / Business Intelligence...,台北市信義區菸廠路88號,40000~0,Business Intelligence business analyst ensures...,,MS SQL Excel Project Tableau Power BI​,\n Basic Requirements- ...,Human Resource,https://m.104.com.tw/job/7agz6,40000,面議
30,2021-05-27,燦坤實業股份有限公司,總部-行銷策略部-CRM數據分析資深專員,台北市內湖區堤頂大道一段331號5樓,38000~50000,【工作職掌】1. CRM數據分析與運用2. CRM操作效益追蹤與提升CRM價值3. 進行會員...,統計軟體操作,,\n 1.主動積極，具數據研究分析熱誠。數據解...,曾先生,https://m.104.com.tw/job/6bcn9,38000,50000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2021-04-23,華碩電腦股份有限公司,AD30084 數據分析運用工程師,台北市北投區 ...,30000~60000,1. 具有網路基礎架構與網路服務常識2. 具有 Window、Linux基本管理能力3. 具...,,hadoop,\n 1. Linux各平台操作(RHEL、C...,人資處招募任用部,https://m.104.com.tw/job/6x5r9,30000,60000
54,2021-04-22,中華電信關係企業_宏華國際股份有限公司,中華電信子公司-數據分析人員,新北市板橋區民族路168號,35000~50000,1.分析業務數據及撰寫簡報。2.協助規劃各項獎金與行銷制度。3.利用資料處理工具製作與整理報...,,Access Excel PowerPoint Power BI​,\n 1.具1年以上數據分析或專案管理相關經驗...,招募任用部,https://m.104.com.tw/job/79ilu,35000,50000
47,2021-04-21,精誠資訊股份有限公司,數據分析顧問(B216),台北市內湖區瑞光路318號 ...,40000~0,工作內容我們是Tableau、Alteryx與Siemens MindSphere等產品的代...,專案規劃執行╱範圍管理 專案溝通╱整合管理 資料庫軟體應用 MES工廠執行系統規劃管理 系統...,Systems Analysis Systems Analyst Data Modeling...,\n,人力資源部,https://m.104.com.tw/job/6n0rh,40000,面議
79,2021-03-29,台灣之星電信股份有限公司,07_用戶行為數據分析師,新北市中和區建一路186號,30000~44000,**40%數據分析1.對發現或挖掘到的問題具有敏銳的觀察力，掌握各種演算法的使用時機2.以開...,,,\n 1. 大學以上畢業2. 熟悉 R 或 P...,人力資源部,https://m.104.com.tw/job/4m2t0,30000,44000


#### 地區
1. 去除所有空格
2. 從work_place中分成出area

In [18]:
df2["work_place"] = df2["work_place"].apply(lambda x: re.sub(" ", "", x))

In [19]:
df2["area"] = df2["work_place"].str.strip().str[:3]

In [20]:
df2["work_place"] = df2["work_place"].str.strip().str[3:]

#### 處理job_skill 、 job_tool資料，除去空格

1. 為空值者，設為不拘 

In [21]:
df2.loc[(df2["job_tool"] == "") | (df2["job_skill"] == ""), ["job_skill", "job_tool"]] = "不拘"

In [22]:
df2.loc[(df2["plus"] == ""), "plus"] = "無"

#### 其他條件(plus)、工作內容(description) => 除去雜質(\s、\n..等等)

In [23]:
df2["plus"] = df2["plus"].apply(lambda x: re.sub(r"\s", "", x)) ## r"\s" => r""表示此字串套用re規則，\s代表匹配任何空白字符

In [24]:
df2["description"] = df2["description"].apply(lambda x: re.sub("\s", "", x))

In [26]:
df2

,update_time,company,title,work_place,salary,description,job_skill,job_tool,plus,recruiter,URL,salary_min,salary_max,area
19,2021-05-26,易遊網股份有限公司,P0700B25-01【數位行銷】數據分析師,中山區民生東路三段49號4樓（距捷運中山國中站500公尺）,40000~0,1.負責網站/App流量監控、釐清數據、資料分析、發掘問題，提供數據分析報告建議與諮詢。2....,不拘,不拘,◎工作技能1.熟悉Web&APP數位行為/流量監測，分析工具：GoogleAnalytics...,易遊網招募中心,https://m.104.com.tw/job/6rglt,40000,面議,台北市
54,2021-05-26,慧邦科技股份有限公司,遊戲數據分析工程師(Data Scientist),中正區（距捷運台大醫院站360公尺）,40000~0,【職務說明】1.數據後台功能開發。2.數據資料庫建置與維護。3.報表排程建置與維護。4.使用...,不拘,不拘,★因配合防疫措施，目前皆採用視訊面試進行。,余小姐,https://m.104.com.tw/job/6u7bu,40000,面議,台北市
24,2021-05-26,財團法人中衛發展中心,FF-數據分析師,中正區杭州南路一段15-1號（距捷運善導寺站420公尺）,40000~0,1.執行產企業數據分析BI或AI輔導。2.完成企業數據分析需求訪談，透過ETL建立數據模型。...,不拘,不拘,1.工業工程、工業管理、資工或資管、機械工程、企管等相關科系碩士，或具備實務生產線經驗者。2...,人力資源,https://m.104.com.tw/job/6th3c,40000,面議,台北市
136,2021-05-26,皮克網路有限公司,【大數據分析工程師】分析萬事萬物！加入台中矽谷,豐原區,30000~52000,1.具備資料庫結構(MySQL)與擷取資料探勘2.熟悉大數據處理相關技術與統計之資料探勘相關...,不拘,不拘,※良好溝通協調能力，善於溝通※思考清晰、具備數據整理能力、統計分析能力,羅先生,https://m.104.com.tw/job/5a5et,30000,52000,台中市
62,2021-05-26,財團法人中國生產力中心,Python工程師(文字探勘、數據分析),汐止區新台五路一段79號2樓(221)汐科站旁,30000,1.專案網站/系統開發及維護2.熟悉Python，同時熟悉C#尤佳3.資料庫操作與管理4.訪...,不拘,不拘,1.熟悉python語言及資料處理套件2.熟悉ASP.NET、C#網頁後端架購，具實際開發經...,潘小姐,https://m.104.com.tw/job/6dhx1,30000,面議,新北市
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2021-04-23,華碩電腦股份有限公司,AD30084 數據分析運用工程師,北投區（距捷運石牌站100公尺）,30000~60000,1.具有網路基礎架構與網路服務常識2.具有Window、Linux基本管理能力3.具有大數據...,不拘,不拘,1.Linux各平台操作(RHEL、CentOS、Ubuntu..etc)3年以上2.大數據...,人資處招募任用部,https://m.104.com.tw/job/6x5r9,30000,60000,台北市
58,2021-04-22,中華電信關係企業_宏華國際股份有限公司,中華電信子公司-數據分析人員,板橋區民族路168號,35000~50000,1.分析業務數據及撰寫簡報。2.協助規劃各項獎金與行銷制度。3.利用資料處理工具製作與整理報...,不拘,不拘,1.具1年以上數據分析或專案管理相關經驗。2.數字邏輯佳，具數據交叉分析相關經驗或會流量分析...,招募任用部,https://m.104.com.tw/job/79ilu,35000,50000,新北市
44,2021-04-21,精誠資訊股份有限公司,數據分析顧問(B216),內湖區瑞光路318號（距捷運港墘站420公尺）,40000~0,工作內容我們是Tableau、Alteryx與SiemensMindSphere等產品的代理...,專案規劃執行╱範圍管理 專案溝通╱整合管理 資料庫軟體應用 MES工廠執行系統規劃管理 系統...,Systems Analysis Systems Analyst Data Modeling...,,人力資源部,https://m.104.com.tw/job/6n0rh,40000,面議,台北市
77,2021-03-29,台灣之星電信股份有限公司,07_用戶行為數據分析師,中和區建一路186號,30000~44000,**40%數據分析1.對發現或挖掘到的問題具有敏銳的觀察力，掌握各種演算法的使用時機2.以開...,不拘,不拘,1.大學以上畢業2.熟悉R或Python與其相關套件佳3.中文打字每分鐘40字以上4.有服務...,人力資源部,https://m.104.com.tw/job/4m2t0,30000,44000,新北市


#### 重新排序columns

In [25]:
col = list(df2.columns)
col

['update_time',
 'company',
 'title',
 'work_place',
 'salary',
 'description',
 'job_skill',
 'job_tool',
 'plus',
 'recruiter',
 'URL',
 'salary_min',
 'salary_max',
 'area']

In [27]:
df2 = df2[col[:3] + [col[-1]] + [col[3]] + col[-3:-1] + col[5:-3]]

In [29]:
df2

,update_time,company,title,area,work_place,salary_min,salary_max,description,job_skill,job_tool,plus,recruiter,URL
0,2021-05-27,大數據股份有限公司,數據分析師,台北市,中山區（距捷運大直站440公尺）,35000,50000,茫茫網海中，誰是KOL關鍵領袖，引領著話題風向？FB數據中，哪個分眾才是主要品牌的目標族群？...,不拘,不拘,有一年以上工作經驗具備資料庫分析經驗擁有不同數據類型的分析經驗瞭解並喜歡研究網路行銷、社群操...,林小姐,https://m.104.com.tw/job/5944x
1,2021-05-27,OB嚴選_橘熊科技股份有限公司,行銷數據分析專員-行銷整合處,台北市,南港區八德路四段768巷1弄18號13樓（距捷運松山站370公尺）,40000,面議,1.數據相關專案規劃與執行。2.因應公司行銷策略進行相關規劃及與跨部門溝通協調達成目標。3....,不拘,不拘,-對於使用行銷自動化工具有高度熱忱(無經驗可)-對電商營運數據(流量/會員/訂單)有基礎的解...,人資部,https://m.104.com.tw/job/7aak3
2,2021-05-27,歐愛兒股份有限公司,(資深) 數據分析師 (儲備幹部),台北市,大安區新生南路三段88號6樓（距捷運公館站380公尺）,40000,面議,1.搜集市場情報及對銷售數字，數據資料清洗與統計分析。2.運用演算法建立模型及優化，整合產出...,財務及營業分析 市場調查資料分析與報告撰寫 系統整合分析 資料庫系統管理維護 資料庫軟體應用,Python MS SQL,無,HR,https://m.104.com.tw/job/77hna
3,2021-05-27,新加坡商蝦皮娛樂電商有限公司台灣分公司,Business Analyst 商業分析師 / Business Intelligence...,台北市,信義區菸廠路88號,40000,面議,BusinessIntelligencebusinessanalystensuresthat...,不拘,不拘,BasicRequirements-Relevantprojectmanagementexp...,Human Resource,https://m.104.com.tw/job/7agz6
4,2021-05-27,燦坤實業股份有限公司,總部-行銷策略部-CRM數據分析資深專員,台北市,內湖區堤頂大道一段331號5樓,38000,50000,【工作職掌】1.CRM數據分析與運用2.CRM操作效益追蹤與提升CRM價值3.進行會員數據蒐...,不拘,不拘,1.主動積極，具數據研究分析熱誠。數據解讀與邏輯能力強2.會員、零售、市調數據分析、點數分析...,曾先生,https://m.104.com.tw/job/6bcn9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2021-04-23,華碩電腦股份有限公司,AD30084 數據分析運用工程師,台北市,北投區（距捷運石牌站100公尺）,30000,60000,1.具有網路基礎架構與網路服務常識2.具有Window、Linux基本管理能力3.具有大數據...,不拘,不拘,1.Linux各平台操作(RHEL、CentOS、Ubuntu..etc)3年以上2.大數據...,人資處招募任用部,https://m.104.com.tw/job/6x5r9
146,2021-04-22,中華電信關係企業_宏華國際股份有限公司,中華電信子公司-數據分析人員,新北市,板橋區民族路168號,35000,50000,1.分析業務數據及撰寫簡報。2.協助規劃各項獎金與行銷制度。3.利用資料處理工具製作與整理報...,不拘,不拘,1.具1年以上數據分析或專案管理相關經驗。2.數字邏輯佳，具數據交叉分析相關經驗或會流量分析...,招募任用部,https://m.104.com.tw/job/79ilu
147,2021-04-21,精誠資訊股份有限公司,數據分析顧問(B216),台北市,內湖區瑞光路318號（距捷運港墘站420公尺）,40000,面議,工作內容我們是Tableau、Alteryx與SiemensMindSphere等產品的代理...,專案規劃執行╱範圍管理 專案溝通╱整合管理 資料庫軟體應用 MES工廠執行系統規劃管理 系統...,Systems Analysis Systems Analyst Data Modeling...,,人力資源部,https://m.104.com.tw/job/6n0rh
148,2021-03-29,台灣之星電信股份有限公司,07_用戶行為數據分析師,新北市,中和區建一路186號,30000,44000,**40%數據分析1.對發現或挖掘到的問題具有敏銳的觀察力，掌握各種演算法的使用時機2.以開...,不拘,不拘,1.大學以上畢業2.熟悉R或Python與其相關套件佳3.中文打字每分鐘40字以上4.有服務...,人力資源部,https://m.104.com.tw/job/4m2t0


#### 重設index

In [28]:
df2.reset_index(drop=True, inplace=True)

#### 存成excel檔

In [30]:
df2.to_excel("./爬蟲_104人力銀行.xlsx", index=False)